In [ ]:
from langchain_groq import ChatGroq
import PyPDF2
import docx
import io
import os

In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0,
    groq_api_key='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'
)

response = llm.invoke("The first person to land on moon was ...")
print(response.content)

In [ ]:
from langchain_community.document_loaders import WebBaseLoader


def load_job_from_url(url):
    loader = WebBaseLoader(url)
    page_data = loader.load().pop().page_content
    return page_data


job_url = "https://jobs.nike.com/job/R-33460"
job_page_data = load_job_from_url(job_url)
print(job_page_data[:500])

In [ ]:
def extract_text_from_pdf(pdf_file_path):
    """Extract text from PDF resume"""
    text = ""
    with open(pdf_file_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text

def extract_text_from_docx(docx_file_path):
    """Extract text from DOCX resume"""
    doc = docx.Document(docx_file_path)
    text = ""
    for paragraph in doc.paragraphs:
        text += paragraph.text + "\n"
    return text

def extract_resume_text(file_path):
    """Extract text from resume file (PDF or DOCX)"""
    if file_path.endswith('.pdf'):
        return extract_text_from_pdf(file_path)
    elif file_path.endswith('.docx'):
        return extract_text_from_docx(file_path)
    else:
        raise ValueError("Unsupported file format. Please use PDF or DOCX.")

# Example usage
resume_path = "AbhinavTirole_resume.pdf"
resume_text = extract_resume_text(resume_path)
print(resume_text[:500])

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt_extract_job = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM JOB POSTING:
    {page_data}
    ### INSTRUCTION:
    The text above is from a job posting or careers page.
    Extract the job information and return it in JSON format containing:
    `role`, `company`, `experience`, `skills`, `description`, and `location`.
    Only return valid JSON.
    ### VALID JSON (NO PREAMBLE):
    """
)

# Example with direct job description
job_description = """
Software Engineer II, AI/ML Platforms at Nike
We are looking for a Software Engineer II to design and implement core components of our data science services and platform and scale it to serve Nike globally. The focus of this engineer will be at the intersection of computer systems and machine learning.

Requirements:
- Bachelor's degree in Computer Science or related field
- 2+ years professional software development experience
- Programming experience with Java, Python, Golang
- Experience with Infrastructure as Code / DevOps practices like Terraform
- Experience with Cloud services (AWS, Azure)
- Experience developing microservices
"""

chain_extract_job = prompt_extract_job | llm
job_res = chain_extract_job.invoke(input={'page_data': job_description})
print(job_res.content)

In [ ]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
job_json = json_parser.parse(job_res.content)
print("Extracted Job Info:")
print(job_json)

In [ ]:
prompt_extract_resume = PromptTemplate.from_template(
    """
    ### RESUME TEXT:
    {resume_text}
    ### INSTRUCTION:
    Extract relevant information from the resume and return it in JSON format containing:
    `name`, `email`, `phone`, `skills`, `experience`, `education`, `projects`, `certifications`.
    Focus on technical skills, years of experience, and relevant projects.
    Only return valid JSON.
    ### VALID JSON (NO PREAMBLE):
    """
)

# Sample resume text for demonstration
sample_resume = """
John Doe
Software Engineer
Email: john.doe@email.com
Phone: +1-234-567-8900

EXPERIENCE:
Senior Software Engineer at TechCorp (2021-2024)
- Developed microservices using Python and Java
- Implemented ML models using TensorFlow and PyTorch
- Worked with AWS services including EC2, S3, Lambda
- Used Docker and Kubernetes for containerization

Software Engineer at StartupXYZ (2019-2021)
- Built web applications using React and Node.js
- Designed RESTful APIs with Express.js
- Worked with MongoDB and PostgreSQL databases

SKILLS:
Programming: Python, Java, JavaScript, Go
ML/AI: TensorFlow, PyTorch, Scikit-learn
Cloud: AWS, Azure, GCP
DevOps: Docker, Kubernetes, Terraform, Jenkins
Databases: MongoDB, PostgreSQL, MySQL

PROJECTS:
- AI-powered recommendation system using Python and TensorFlow
- Microservices architecture for e-commerce platform
- Real-time data processing pipeline using Apache Kafka
"""

chain_extract_resume = prompt_extract_resume | llm
resume_res = chain_extract_resume.invoke(input={'resume_text': sample_resume})
resume_json = json_parser.parse(resume_res.content)
print("Extracted Resume Info:")
print(resume_json)

In [ ]:
import pandas as pd


portfolio_data = {
    'Project': [
        'AI Recommendation Engine',
        'Microservices E-commerce Platform',
        'Real-time Analytics Dashboard',
        'Machine Learning Model Deployment',
        'DevOps CI/CD Pipeline',
        'Cloud Infrastructure Automation',
        'Mobile App Backend API',
        'Data Processing Pipeline',
        'Web Application with React',
        'Computer Vision Project'
    ],
    'Techstack': [
        'Python, TensorFlow, AWS, Docker',
        'Java, Spring Boot, Kubernetes, PostgreSQL',
        'React, Node.js, MongoDB, Socket.io',
        'Python, MLflow, AWS SageMaker, FastAPI',
        'Jenkins, Docker, Terraform, AWS',
        'Terraform, AWS CloudFormation, Python',
        'Node.js, Express, MongoDB, JWT',
        'Apache Kafka, Python, Elasticsearch',
        'React, Redux, Node.js, MySQL',
        'Python, OpenCV, TensorFlow, Flask'
    ],
    'Links': [
        'https://github.com/johndoe/ai-recommendation',
        'https://github.com/johndoe/microservices-ecommerce',
        'https://github.com/johndoe/realtime-dashboard',
        'https://github.com/johndoe/ml-deployment',
        'https://github.com/johndoe/devops-pipeline',
        'https://github.com/johndoe/cloud-automation',
        'https://github.com/johndoe/mobile-backend',
        'https://github.com/johndoe/data-pipeline',
        'https://github.com/johndoe/react-webapp',
        'https://github.com/johndoe/computer-vision'
    ]
}

df = pd.DataFrame(portfolio_data)
print("Personal Projects Portfolio:")
print(df)

In [ ]:
import uuid
import chromadb

client = chromadb.PersistentClient('candidate_vectorstore')
collection = client.get_or_create_collection(name="personal_projects")

# Clear existing data and add new portfolio
collection.delete(where={})

for _, row in df.iterrows():
    collection.add(
        documents=row["Techstack"],
        metadatas={"project": row["Project"], "links": row["Links"]},
        ids=[str(uuid.uuid4())]
    )

print(f"Added {collection.count()} projects to vector store")

In [ ]:
# Extract skills from job posting
job_skills = job_json.get('skills', [])
print("Job Requirements:")
for skill in job_skills:
    print(f"- {skill}")

# Query vector store for relevant projects
relevant_projects = collection.query(
    query_texts=job_skills,
    n_results=3
).get('metadatas', [])

print("\nRelevant Projects Found:")
for project_list in relevant_projects:
    for project in project_list:
        print(f"- {project['project']}: {project['links']}")

In [ ]:
prompt_email = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION:
    {job_description}

    ### CANDIDATE INFORMATION:
    {candidate_info}

    ### RELEVANT PROJECTS:
    {relevant_projects}

    ### INSTRUCTION:
    You are writing a cold email from a job candidate to a hiring manager/recruiter.
    The candidate is applying for the job mentioned above.

    Write a professional, personalized cold email that:
    1. Shows genuine interest in the role and company
    2. Highlights relevant experience and skills that match the job requirements
    3. Mentions 2-3 most relevant projects with links
    4. Demonstrates knowledge about the company/role
    5. Requests an interview or conversation
    6. Uses a professional but engaging tone

    Keep it concise (under 200 words) and compelling.
    Use the candidate's name from their information.

    ### EMAIL (NO PREAMBLE):
    """
)

# Prepare data for email generation
candidate_name = resume_json.get('name', 'John Doe')
candidate_email = resume_json.get('email', 'john.doe@email.com')

chain_email = prompt_email | llm
email_res = chain_email.invoke({
    "job_description": str(job_json),
    "candidate_info": str(resume_json),
    "relevant_projects": str(relevant_projects)
})

print("Generated Cold Email:")
print("=" * 50)
print(email_res.content)

In [ ]:
prompt_formal_email = PromptTemplate.from_template(
    """
    ### JOB POSTING:
    Role: {role}
    Company: {company}
    Requirements: {requirements}

    ### CANDIDATE:
    Name: {name}
    Experience: {experience}
    Key Skills: {skills}

    ### PROJECTS:
    {projects}

    ### INSTRUCTION:
    Write a formal cold email for a job application. Include:
    - Professional subject line
    - Formal greeting
    - Brief introduction and interest in the role
    - 2-3 key qualifications that match the job
    - Mention of relevant projects (max 2)
    - Call to action for interview
    - Professional closing

    Format as a complete email with subject line.

    ### FORMAL EMAIL:
    """
)

# Extract specific information for formal email
role = job_json.get('role', 'Software Engineer')
company = job_json.get('company', 'Nike')
requirements = job_json.get('skills', [])[:3]  # Top 3 requirements
name = resume_json.get('name', 'John Doe')
experience = resume_json.get('experience', 'Software Engineer with 3+ years')
skills = resume_json.get('skills', [])[:5]  # Top 5 skills

# Get top 2 relevant projects
top_projects = []
if relevant_projects:
    for project_list in relevant_projects[:2]:
        for project in project_list:
            top_projects.append(f"{project['project']} ({project['links']})")

chain_formal_email = prompt_formal_email | llm
formal_email_res = chain_formal_email.invoke({
    "role": role,
    "company": company,
    "requirements": requirements,
    "name": name,
    "experience": experience,
    "skills": skills,
    "projects": top_projects
})

print("Formal Cold Email:")
print("=" * 50)
print(formal_email_res.content)

In [ ]:
def generate_job_application_email(job_input, resume_text, email_type="casual"):
    """
    Complete pipeline to generate job application email

    Args:
        job_input: Job description text or URL
        resume_text: Extracted resume text
        email_type: "casual" or "formal"

    Returns:
        Generated email content
    """

    # Extract job information
    job_res = chain_extract_job.invoke(input={'page_data': job_input})
    job_data = json_parser.parse(job_res.content)

    # Extract resume information
    resume_res = chain_extract_resume.invoke(input={'resume_text': resume_text})
    resume_data = json_parser.parse(resume_res.content)

    # Find relevant projects
    job_skills = job_data.get('skills', [])
    projects = collection.query(query_texts=job_skills, n_results=3).get('metadatas', [])

    # Generate email
    if email_type == "formal":
        email_result = chain_formal_email.invoke({
            "role": job_data.get('role', ''),
            "company": job_data.get('company', ''),
            "requirements": job_skills[:3],
            "name": resume_data.get('name', ''),
            "experience": resume_data.get('experience', ''),
            "skills": resume_data.get('skills', [])[:5],
            "projects": projects
        })
    else:
        email_result = chain_email.invoke({
            "job_description": str(job_data),
            "candidate_info": str(resume_data),
            "relevant_projects": str(projects)
        })

    return email_result.content

# Example usage
sample_job = """
Senior Python Developer at Google
Looking for experienced Python developer to join our AI team.
Requirements: 5+ years Python, Machine Learning, AWS, Docker
"""

generated_email = generate_job_application_email(sample_job, sample_resume, "casual")
print("Generated Email using Pipeline:")
print("=" * 50)
print(generated_email)